In [79]:
# write your codes here
import torch
import torch.nn as nn
import torch.utils.data as data_utils

from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import glob
import scipy.io
from PIL import Image
import torchvision.models as models
from torch.optim import lr_scheduler
import os
import numpy as np
import h5py
import pandas as pd

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as DD
import torchnet as tnt

import time

# MAT files expected to already generated and placed in the MAT_FILE_PATH below.


In [80]:

class1_path = './UCF101_release/images_class1/*'
class1_path_new = './UCF101_release/images_class1/'

vgg_videos = './UCF101_release/vgg16_relu6/'

mat_file_path = './UCF101_release/vgg16_relu6/'

frame_names = [name[-8:] for name in glob.glob(class1_path )]
image_class1_paths = [name for name in glob.glob(class1_path)]
print(len(frame_names))
#print(frame_names)
#print(image_class1_paths)

0


## Evaluation (LSTM)

In [81]:
# LSTM classification model
class LSTMClassify(nn.Module):
    def __init__(self):
        super(LSTMClassify, self).__init__()
        
        
        ## PREVIOUS ATTEMPTS ###
        '''
        self.recurrent_layer = nn.LSTM(input_size=4096, hidden_size=1200, num_layers=2, batch_first=True, bidirectional=True, dropout=0.25)
        self.classify_layer = nn.Linear(in_features=1200*2, out_features=15)

        self.recurrent_layer = nn.LSTM(input_size=4096, hidden_size=1200, num_layers=2, batch_first=True, bidirectional=True, dropout=0.25)
        self.classify_layer = nn.Linear(in_features=1200*2, out_features=15)
        
        self.recurrent_layer = nn.LSTM(input_size=4096, hidden_size=1200, num_layers=2, batch_first=True, dropout=0.25)
        self.classify_layer = nn.Linear(in_features=1200, out_features=15) 
       
        '''
        ############## Network Architecture ##############
         ###################################################
        ## FINAL ::changed num hidden layers to 1500
        self.recurrent_layer = nn.LSTM(input_size=4096, hidden_size=1500, num_layers=2, 
                                       batch_first=True, bidirectional=True, dropout=0.25)
        self.regression_layer = nn.Linear(in_features=1500*2, out_features=1) ##  question talks about taking only first 15 classes
        ###################################################
    
    # the size of input is [batch_size, seq_len(25), input_dim(4096)]
    # the size of logits is [batch_size, num_class]
    
    def forward(self, input, h_t_1=None, c_t_1=None):
        
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        rnn_outputs, (hn, cn) = self.recurrent_layer(input)
        
        # classify the last step of rnn_outpus
        # the size of logits is [batch_size, num_class]
        logits = self.regression_layer(rnn_outputs[:,-1])
        return logits ## no classification here

model_lstm = LSTMClassify() 


#### optimizers and loss functions for custom classifier
optimizer = torch.optim.Adam(params=model_lstm.parameters(), lr=3e-3, betas=(0.1, 0.2))
#optimizer = torch.optim.SGD(model_lstm.parameters(), lr = 0.0002, momentum=0.95) ## try SGD
criterion = nn.MSELoss()
#num_class = 15

In [82]:

#mat_file_path = './UCF101_release/vgg16_relu6/'
#vls_path = './UCF101_release/annos/videos_labels_subsets.txt'
train_mat_path = '../data/train/mat/'
test_mat_path = '../data/test/mat/'
train_csv_path = '../data/train/files/train1.csv'
test_csv_path = '../data/test/files/driving.csv'


train_df = pd.read_csv(train_csv_path)
print('----------------- TRAIN DF --------------------------')
train_df.head(5)


----------------- TRAIN DF --------------------------


,image_path,frame,speed
0,../data/train/imgs/0.jpg,0,28.105569
1,../data/train/imgs/1.jpg,1,28.105569
2,../data/train/imgs/2.jpg,2,28.106527
3,../data/train/imgs/3.jpg,3,28.130404
4,../data/train/imgs/4.jpg,4,28.109243


In [83]:

test_df = pd.read_csv(test_csv_path)
print('----------------- TEST DF --------------------------')
test_df.head(5)

----------------- TEST DF --------------------------


,image_path,frame,speed
0,../data/test/imgs/0.jpg,0,2.022715
1,../data/test/imgs/1.jpg,1,2.040872
2,../data/test/imgs/2.jpg,2,2.062394
3,../data/test/imgs/3.jpg,3,2.076283
4,../data/test/imgs/4.jpg,4,2.077074


In [84]:

train_features = list()
train_lbls = list()

val_features = list()
val_lbls = list()


test_features = list()
test_lbls = list()

for idx, item in train_df.iterrows():
    print('idx >> ', idx)
    
    #print('item[speed] >> ', item['speed'])
    #print('item[frame] >> ', item['frame'])
    
    try:
        trn_mat = scipy.io.loadmat(train_mat_path + str(item['frame']) , mat_dtype=True)
        trn_feature = np.array(trn_mat['Feature'])
        #print(trn_feature)
        if int(item['frame']) < 16320:
            train_features.append(trn_feature)
            train_lbls.append(float(item['speed']))
        else:
            val_features.append(trn_feature)
            val_lbls.append(float(item['speed']))
    except:
        pass
    
    
    #print(float(item['speed']))
    
print('train length ', len(train_lbls))
print('val length ', len(val_lbls))

('idx >> ', 0)
('idx >> ', 1)
('idx >> ', 2)
('idx >> ', 3)
('idx >> ', 4)
('idx >> ', 5)
('idx >> ', 6)
('idx >> ', 7)
('idx >> ', 8)
('idx >> ', 9)
('idx >> ', 10)
('idx >> ', 11)
('idx >> ', 12)
('idx >> ', 13)
('idx >> ', 14)
('idx >> ', 15)
('idx >> ', 16)
('idx >> ', 17)
('idx >> ', 18)
('idx >> ', 19)
('idx >> ', 20)
('idx >> ', 21)
('idx >> ', 22)
('idx >> ', 23)
('idx >> ', 24)
('idx >> ', 25)
('idx >> ', 26)
('idx >> ', 27)
('idx >> ', 28)
('idx >> ', 29)
('idx >> ', 30)
('idx >> ', 31)
('idx >> ', 32)
('idx >> ', 33)
('idx >> ', 34)
('idx >> ', 35)
('idx >> ', 36)
('idx >> ', 37)
('idx >> ', 38)
('idx >> ', 39)
('idx >> ', 40)
('idx >> ', 41)
('idx >> ', 42)
('idx >> ', 43)
('idx >> ', 44)
('idx >> ', 45)
('idx >> ', 46)
('idx >> ', 47)
('idx >> ', 48)
('idx >> ', 49)
('idx >> ', 50)
('idx >> ', 51)
('idx >> ', 52)
('idx >> ', 53)
('idx >> ', 54)
('idx >> ', 55)
('idx >> ', 56)
('idx >> ', 57)
('idx >> ', 58)
('idx >> ', 59)
('idx >> ', 60)
('idx >> ', 61)
('idx >> ', 62)
('

In [85]:
for idx, item in test_df.iterrows():
    print('idx >> ', idx)
    #print('item[speed] >> ', item['speed'])
    #print('item[frame] >> ', item['frame'])
    
    try:
        tst_mat = scipy.io.loadmat(test_mat_path + str(item['frame']) )
        test_feature = np.array(tst_mat['Feature'])
        test_features.append(test_feature)
        test_lbls.append(float(item['speed']))
    except:
        pass


print('test length ', len(test_lbls))

('idx >> ', 0)
('idx >> ', 1)
('idx >> ', 2)
('idx >> ', 3)
('idx >> ', 4)
('idx >> ', 5)
('idx >> ', 6)
('idx >> ', 7)
('idx >> ', 8)
('idx >> ', 9)
('idx >> ', 10)
('idx >> ', 11)
('idx >> ', 12)
('idx >> ', 13)
('idx >> ', 14)
('idx >> ', 15)
('idx >> ', 16)
('idx >> ', 17)
('idx >> ', 18)
('idx >> ', 19)
('idx >> ', 20)
('idx >> ', 21)
('idx >> ', 22)
('idx >> ', 23)
('idx >> ', 24)
('idx >> ', 25)
('idx >> ', 26)
('idx >> ', 27)
('idx >> ', 28)
('idx >> ', 29)
('idx >> ', 30)
('idx >> ', 31)
('idx >> ', 32)
('idx >> ', 33)
('idx >> ', 34)
('idx >> ', 35)
('idx >> ', 36)
('idx >> ', 37)
('idx >> ', 38)
('idx >> ', 39)
('idx >> ', 40)
('idx >> ', 41)
('idx >> ', 42)
('idx >> ', 43)
('idx >> ', 44)
('idx >> ', 45)
('idx >> ', 46)
('idx >> ', 47)
('idx >> ', 48)
('idx >> ', 49)
('idx >> ', 50)
('idx >> ', 51)
('idx >> ', 52)
('idx >> ', 53)
('idx >> ', 54)
('idx >> ', 55)
('idx >> ', 56)
('idx >> ', 57)
('idx >> ', 58)
('idx >> ', 59)
('idx >> ', 60)
('idx >> ', 61)
('idx >> ', 62)
('

In [86]:
print('test length ', len(test_lbls))

('test length ', 8616)


In [88]:
# tried batch_size = 5, 15
train = DD.TensorDataset(torch.Tensor(np.asarray(train_features)), torch.Tensor(np.asarray(train_lbls)))
train_loader = DD.DataLoader(train, batch_size=15, shuffle=False, sampler=DD.sampler.RandomSampler(train),num_workers=2, pin_memory=False)

val = DD.TensorDataset(torch.Tensor(np.asarray(val_features)), torch.Tensor(np.asarray(val_lbls)))
val_loader = DD.DataLoader(val, batch_size=15, shuffle=False, sampler=DD.sampler.SequentialSampler(val),num_workers=2, pin_memory=False)


In [89]:
use_cuda = torch.cuda.is_available()
print('use cuda: %s'%(use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor

use cuda: False


In [ ]:
# run the model for one epoch
# can be used for both training or validation model
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    #acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()
    
    #print(enumerate(data_loader))
    for batch_idx, sample in enumerate(data_loader):
        #print(batch_idx)
        #print('feature ',sample[0])
        #print('lbl ',sample[1])
        #sequence = sample['seq']
        #label = sample['label']
        
        
        input_sequence_var = sample[0] #Variable(sequence).type(FloatTensor) ## mat file as ftrs
        input_label_var =  Variable(sample[1]).type(FloatTensor) ## speed
        labels=input_label_var.view(-1,1)

        # compute output
        # output_logits: [batch_size, num_class]
        #print("input_sequence_var >> ",input_sequence_var.shape)
        output_speed = model(input_sequence_var)
        #output_speed=output_speed.view(-1,1)
        #print('output_speed ',output_speed.shape[0])
        #print('labels ',labels.shape[0])
        loss = criterion(output_speed, labels)
        
        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.data[0])
        #print("loss >>> ",loss.data[0])
        print('%s Epoch: %d  ,Batch ID: %d, Loss: %.4f'%(logger_prefix, epoch, batch_idx, meter_loss.value()[0]))
        #acc.add(output_speed.data.flatten(), labels.data.flatten())

    print('Final training loss..................')
    print('%s Epoch: %d  ,Batch ID: %d, Loss: %.4f'%(logger_prefix, epoch, batch_idx, meter_loss.value()[0]))
    return meter_loss.value()[0]

start_time = time.time()
num_epochs = 5
evaluate_every_epoch = 5
for e in range(num_epochs):
    mse = run_epoch(train_loader, model_lstm, criterion, e, True, optimizer)
    print('Train MSE using custom LSTM model on this dataset for epoch %s is: %s '%(e,mse))
    if e % evaluate_every_epoch == 0:
        run_epoch(val_loader, model_lstm, criterion, e, False, None)   

end_time = time.time() - start_time
print('------------------------------------------------------------')
print("time taken to TRAIN for LSTM Model  in seconds {:.3f}".format(end_time))

/home/shayan_ray/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:42: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


train Epoch: 0  ,Batch ID: 0, Loss: 239.3412
train Epoch: 0  ,Batch ID: 1, Loss: 161.1051
train Epoch: 0  ,Batch ID: 2, Loss: 132.2174
train Epoch: 0  ,Batch ID: 3, Loss: 114.3302
train Epoch: 0  ,Batch ID: 4, Loss: 105.3168
train Epoch: 0  ,Batch ID: 5, Loss: 97.9263
train Epoch: 0  ,Batch ID: 6, Loss: 92.2852
train Epoch: 0  ,Batch ID: 7, Loss: 93.3548
train Epoch: 0  ,Batch ID: 8, Loss: 91.6965
train Epoch: 0  ,Batch ID: 9, Loss: 90.5844
train Epoch: 0  ,Batch ID: 10, Loss: 89.0496
train Epoch: 0  ,Batch ID: 11, Loss: 86.5171
train Epoch: 0  ,Batch ID: 12, Loss: 85.1284
train Epoch: 0  ,Batch ID: 13, Loss: 85.4411
train Epoch: 0  ,Batch ID: 14, Loss: 83.0748
train Epoch: 0  ,Batch ID: 15, Loss: 81.6838
train Epoch: 0  ,Batch ID: 16, Loss: 81.1097
train Epoch: 0  ,Batch ID: 17, Loss: 77.4233
train Epoch: 0  ,Batch ID: 18, Loss: 74.8603
train Epoch: 0  ,Batch ID: 19, Loss: 79.0897
train Epoch: 0  ,Batch ID: 20, Loss: 84.0664
train Epoch: 0  ,Batch ID: 21, Loss: 82.5048
train Epoch: 0 

In [ ]:

testds = DD.TensorDataset(torch.Tensor(np.asarray(test_features)), torch.Tensor(np.asarray(test_lbls)))
test_loader = DD.DataLoader(testds, batch_size=5, shuffle=False, sampler=DD.sampler.SequentialSampler(testds),num_workers=2, pin_memory=False)

In [ ]:
def predict_on_test(model_lstm, data_loader):
    model_lstm.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    
    #acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()
    
    #print(enumerate(data_loader))
    for batch_idx, sample in enumerate(data_loader):
        #print(batch_idx)
        #print('feature ',sample[0])
        #print('lbl ',sample[1])
        #sequence = sample['seq']
        #label = sample['label']
        
        
        input_sequence_var = sample[0] #Variable(sequence).type(FloatTensor) ## mat file as ftrs
        input_label_var =  Variable(sample[1]).type(FloatTensor) ## speed
        labels=input_label_var.view(-1,1)

        # compute output
        # output_logits: [batch_size, num_class]
        #print("input_sequence_var >> ",input_sequence_var.shape)
        output_speed = model(input_sequence_var)
        print('output_speed ',output_speed.shape)
        print(torch.max(output_speed,1))
        #output_speed=output_speed.view(-1,1)
        print('output_speed ',output_speed.shape)
        #print('labels ',labels.shape[0])
        loss = criterion(output_speed, labels)
        
        meter_loss.add(loss.data[0])
        #print("loss >>> ",loss.data[0])
        print('%s Batch ID: %d, Loss: %.4f'%(logger_prefix,  batch_idx, meter_loss.value()[0]))
        #acc.add(output_speed.data.flatten(), labels.data.flatten())

    print('Final Testing loss..................')
    print('%s Epoch: %d  ,Batch ID: %d, Loss: %.4f'%(logger_prefix, epoch, batch_idx, meter_loss.value()[0]))
    return meter_loss.value()[0]
    '''
    
    count=0
    
    for batch_idx, sample in enumerate(data_loader):
        input_sequence_var = sample[0] #Variable(sequence).type(FloatTensor)
        actual_label_var =  Variable(sample[1]).type(LongTensor)
        scores = model_lstm(input_sequence_var)
        print(torch.max(scores,1))
        _, preds = torch.max(scores,1) #scores.data.max(1)
        #print("preds %s  ",preds)
        #print("actual_label_var ",actual_label_var)
        acc.add(scores.data, actual_label_var.data)
    return acc.value()[0]
    '''
    
start_time = time.time()
mse=predict_on_test(model_lstm, test_loader)
print('Test MSE using custom LSTM model on this dataset is: ',mse)
end_time = time.time() - start_time 
print('------------------------------------------------------------')
print("time taken to Predict TEST data  in seconds {:.3f}".format(end_time))

## Q2. Evaluation (SVM)

In [ ]:

train_features_svm = list()
train_lbls_svm = list()

test_features_svm = list()
test_lbls_svm = list()

for idx, item in train_df.iterrows():
    print('idx >> ', idx)
    
    #print('item[speed] >> ', item['speed'])
    #print('item[frame] >> ', item['frame'])
    
    try:
        trn_mat = scipy.io.loadmat(train_mat_path + str(item['frame']) , mat_dtype=True)
        trn_feature = np.array(trn_mat['Feature']).flatten()
        #print(trn_feature)
        train_features_svm.append(trn_feature)
        train_lbls_svm.append(float(item['speed']))
    except:
        pass
    
    
    #print(float(item['speed']))
    
print('train length svm', len(train_lbls_svm))


In [ ]:
for idx, item in test_df.iterrows():
    print('idx >> ', idx)
    #print('item[speed] >> ', item['speed'])
    #print('item[frame] >> ', item['frame'])
    
    try:
        tst_mat = scipy.io.loadmat(test_mat_path + str(item['frame']) )
        test_feature = np.array(tst_mat['Feature']).flatten()
        test_features_svm.append(test_feature)
        test_lbls_svm.append(float(item['speed']))
    except:
        pass


print('test length ', len(test_lbls_svm))

In [ ]:
#### SVM Training 
from sklearn import svm


## PREVIOUS ATTEMPTS ###
'''
model_svm = svm.LinearSVC(C=0.091, tol=5e-3, loss='hinge', penalty='l2', random_state=42)
model_svm = svm.SVC(kernel='poly', C=0.75, tol=5e-3)
model_svm = svm.LinearSVC(C=5)
model_svm = svm.SVC(C=0.091, tol=5e-3, kernel='rbf')
model_svm = svm.SVC(C=50)
'''
start_time = time.time()
model_svm = svm.SVR(kernel='rbf', C=1e3, gamma=0.1)
regrsr = model_svm.fit(np.array(train_features_svm), np.array(train_lbls_svm))
print('------------------------------------------------------------')
regrsr.score(np.array(train_features_svm), np.array(train_lbls_svm))
end_time = time.time() - start_time 
print('------------------------------------------------------------')
print("time taken to TRAIN SVM  in seconds {:.3f}".format(end_time))

In [ ]:
start_time = time.time()
preds_svm = model_svm.predict(np.array(test_features_svm))
#print(preds_svm.shape)
test_mse = mean_squared_error(preds_svm, test_lbls_svm)
print('Test MSE for SVM is ',test_mse)
end_time = time.time() - start_time 
print('------------------------------------------------------------')
print("time taken to PREDICT with SVM  in seconds {:.3f}".format(end_time))